# 応用情報工学演習：Scikit-Learnマスターコース
## 実践編 配布用コードファイル

#### 基本ライブラリの読み込み

In [1]:
import os, sys
import glob

import warnings
warnings.simplefilter('ignore')

import numpy as np
from sklearn.metrics import accuracy_score
from PIL import Image

# 自作モジュールのインポート
from utils.train import fit, evaluate_history, generate_pseudo_labels ,torch_seed, fit_self_train
from utils.dataset import MyDataset

# 必要ライブラリのインポート
import matplotlib.pyplot as plt

# PyTorch関連ライブラリのインポート
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models
import torchvision.transforms as transforms

# warning表示off
import warnings
warnings.simplefilter('ignore')

# デフォルトフォントサイズ変更
plt.rcParams['font.size'] = 14

# デフォルトグラフサイズ変更
plt.rcParams['figure.figsize'] = (6,6)

# デフォルトで方眼表示ON
plt.rcParams['axes.grid'] = True

# numpyの表示桁数設定
np.set_printoptions(suppress=True, precision=5)

np.random.seed(1)

### Cudaのチェック

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


#### データ読み込み関数

In [3]:
#-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-#
#                               このセルは変更を禁止します                                  #
#-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-#
#-------------------------------------------------------------------------------------------#
#-                                  データ読み込み関数                                     -#
#-------------------------------------------------------------------------------------------#
def load_data_(root_dir='../data/gw_dataset'):
    """
    グループワーク用　学習用データ読み込み関数

    Args:
        * root_dir: 配布したgw_datasetへのパスを指定する (default: '../data/gw_dataset')
        
    Output:
        * X_train:  正解ラベル付き学習用画像   shape: (140x150,528)
                    1行あたり、画像1枚の画素値を格納したnp.array
                    画像140枚、各画像224x224=150,528次元のベクトル
                    
        * y_train:  X_trainの正解ラベル        shape: (140, )
                    クラス数は7　140枚分のクラスラベル（0～6）の整数が格納されている
        
        * X_trainu: 正解ラベルなし学習用画像   shape: (210x150,528)
                    形式はX_trainと同じ
                    
        * X_val:   正解ラベル付き検証用画像   shape: (70x150,528)
                    形式はX_trainと同じ
                    
        * y_val:    X_valの正解ラベル          shape: (70, )
                    形式はy_valと同じ                             
    """
    
    #----------------- 学習用データの読み込み -------------------#
    train_paths = sorted(glob.glob(os.path.join(root_dir, "train", "*.png")))
    
    X_train = np.array([np.array(Image.open(p)).astype(np.float32).ravel() for p in train_paths])
    y_train = np.load(os.path.join(root_dir, "y_train.npy"))

    #------------ ラベルなし学習用データの読み込み --------------#
    trainu_paths = sorted(glob.glob(os.path.join(root_dir, "train-u", "*.png")))
    X_trainu = np.array([np.array(Image.open(p)).astype(np.float32).ravel() for p in trainu_paths])
    
    #----------------- 検証用データの読み込み -------------------#
    val_paths = sorted(glob.glob(os.path.join(root_dir,"val", "*.png")))
    X_val = np.array([np.array(Image.open(p)).astype(np.float32).ravel() for p in val_paths])
    y_val = np.load(os.path.join(root_dir, "y_val.npy"))

    return X_train, y_train, X_trainu, X_val, y_val

In [4]:
### データ読み込み関数の実行
X_train, y_train, X_trainu, X_val, y_val = load_data_() # すべて必要な場合
print(X_train.shape, y_train.shape, X_trainu.shape, X_val.shape, y_val.shape)

# X_train, y_train, _, X_val, y_val = load_data_() # いらないデータがある場合
# print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(140, 150528) (140,) (210, 150528) (70, 150528) (70,)


#### 評価関数

In [5]:
#-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-#
#                               このセルは変更を禁止します                                  #
#-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-#
#-------------------------------------------------------------------------------------------#
#-                                       評価関数                                          -#
#-------------------------------------------------------------------------------------------#

def eval_(y_pred):
    """
    グループワーク用　評価関数
    実行する前にy_valがglobalスコープに読み込まれている必要がある

    Args:
        * y_pred:   識別結果　y_valと同形式・同shapeでなければならない
    """

    try:
        y_val
    except NameError as e:
        print("y_valが読み込まれていません")
        
    assert y_pred.shape == y_val.shape, 'y_predとy_valのサイズが一致しません'
    
    print("valデータでの識別精度:{0:.3f}".format(accuracy_score(y_val, y_pred)))

### データの確認

In [6]:
# 分類先クラスのリスト作成
classes = [ '0', '1', '2', '3', '4', '5', '6']

# データを復元
X_train_reshaped = X_train.reshape(-1, 224, 224, 3)
X_val_reshaped = X_val.reshape(-1, 224, 224, 3)
X_trainu_reshaped = X_trainu.reshape(-1, 224, 224, 3)

# データセット作成
train_data = MyDataset(X_train_reshaped, y_train, type='no_trans')
val_data = MyDataset(X_val_reshaped, y_val, type='val')
trainu_data = MyDataset(X_trainu_reshaped, None, type='no_trans')

# データ件数確認
print(f'訓練データ: {len(train_data)}件')
print(f'ラベルなし訓練データ: {len(trainu_data)}件')
print(f'検証データ: {len(val_data)}件')

# データローダー作成
batch_size = 70
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
unlabeled_loader = DataLoader(trainu_data, batch_size=batch_size, shuffle=True)


訓練データ: 140件
ラベルなし訓練データ: 210件
検証データ: 70件


### モデルの定義

In [7]:
# 試行回数
num_epochs = 1000

# Consistency regularizationの重み
w = 0

# historyファイルを初期化する
history = np.zeros((0, 5), dtype=float)

# 学習モデル定義
net = models.resnet18(pretrained = True)
net.fc = nn.Linear(in_features=net.fc.in_features, out_features=7)  # 7クラスに変更

# 乱数初期化
torch_seed()

# GPUの利用
net = net.to(device)

# 学習率
lr = 1e-4

# 損失関数定義
ce = nn.CrossEntropyLoss()
mse = nn.MSELoss()

# 最適化関数定義
optimizer = optim.Adam(net.parameters(), lr=lr)

### Π-model

In [8]:
weak_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

strong_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandAugment(num_ops=2, magnitude=10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:
history = np.zeros((0, 5), dtype=float)
base_epochs = len(history)
net.train()

for epoch in range(base_epochs, num_epochs + base_epochs):
    train_loss = 0.0
    train_acc = 0.0
    val_loss = 0.0
    val_acc = 0.0
    
    for (x_l, y_l), x_u in zip(train_loader, unlabeled_loader):
        # 全てGPUへ転送
        x_l, y_l, x_u = x_l.to(device), y_l.to(device), x_u.to(device)
        
        # ラベルありデータのデータ拡張
        x_l_weak = torch.stack([weak_transform(x_l[i].cpu()) for i in range(x_l.size(0))]).to(device)
        x_l_strong = torch.stack([strong_transform(x_l[i].cpu()) for i in range(x_l.size(0))]).to(device)
        
        # ラベルあり拡張データとラベルなしデータを結合
        x_weak = torch.cat([x_l_weak, x_u], dim=0)
        x_strong = torch.cat([x_l_strong, x_u], dim=0)

        # ネットワークで学習する
        z_weak = net(x_weak)
        with torch.no_grad():
            z_strong = net(x_strong)
        
        # weak_transform の出力を教師ありデータの数分スライス
        z_l_weak = z_weak[:x_l.size(0)]  # ラベルあり部分の出力
        y_pred = z_l_weak.max(1)[1]  # 予測ラベル
        
        # Lossの計算
        Lx = ce(z_l_weak, y_l)
        Lu = mse(z_weak, z_strong)

        # 損失の合計
        loss = Lx + w * Lu

        # 損失と精度の記録
        train_loss += loss.item()
        train_acc += (y_pred == y_l).sum().item()
        
        # 逆伝播と最適化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # wの更新
    w = min(1, w + 0.0001)

    # 検証データでの損失と精度計算
    net.eval()
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            loss = ce(outputs, labels)
            val_loss += loss.item()
            _, preds = outputs.max(1)
            val_acc += (preds == labels).sum().item()

    # 平均損失と精度の計算
    avg_train_loss = train_loss / len(train_loader.dataset)
    avg_train_acc = train_acc / len(train_loader.dataset)
    avg_val_loss = val_loss / len(val_loader.dataset)
    avg_val_acc = val_acc / len(val_loader.dataset)

    # エポック結果表示
    if epoch % 50 == 0:
        print(f"Epoch {epoch+1}/{num_epochs+base_epochs}: "
              f"Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f}, "
              f"Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc:.4f}")

    # 履歴更新
    history = np.vstack((history, [epoch + 1, avg_train_loss, avg_train_acc, avg_val_loss, avg_val_acc]))


Epoch 1/1000: Train Loss: 0.0290, Train Acc: 0.1429, Val Loss: 0.0259, Val Acc: 0.2714
Epoch 51/1000: Train Loss: 0.0001, Train Acc: 1.0000, Val Loss: 0.0093, Val Acc: 0.7571
Epoch 101/1000: Train Loss: 0.0002, Train Acc: 1.0000, Val Loss: 0.0100, Val Acc: 0.7286
Epoch 151/1000: Train Loss: 0.0536, Train Acc: 0.7929, Val Loss: 0.0392, Val Acc: 0.4000
Epoch 201/1000: Train Loss: 0.0016, Train Acc: 1.0000, Val Loss: 0.0392, Val Acc: 0.7143
Epoch 251/1000: Train Loss: 0.0023, Train Acc: 1.0000, Val Loss: 0.0174, Val Acc: 0.7286
Epoch 301/1000: Train Loss: 0.0032, Train Acc: 1.0000, Val Loss: 0.0140, Val Acc: 0.6857
Epoch 351/1000: Train Loss: 224109218388548681728.0000, Train Acc: 0.1429, Val Loss: 34023074318.6286, Val Acc: 0.1429
Epoch 401/1000: Train Loss: 236248799858400952320.0000, Train Acc: 0.1429, Val Loss: 43487820887.7714, Val Acc: 0.1429
Epoch 451/1000: Train Loss: 551861608610194325504.0000, Train Acc: 0.1429, Val Loss: 47470503526.4000, Val Acc: 0.1429
Epoch 501/1000: Train L

In [ ]:
# 結果確認
evaluate_history(history)

### モデルの保存

In [ ]:
path = 'model.pth'
torch.save(net.state_dict(), path)